In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import *
import copy
import cv2

In [ ]:
def KT(amp_A,width_A,pos_A,amp_I,width_I,pos_I):
    grid = 400
    length = 200
    kernel_length = 40
    dx = length/grid
    dt = 0.1
    max_S = 2
    deg = 0.1


    loopNumber = 1
    stepLoop = 3000

    class Gaussian_Func():
        def __init__(self,amp,width,pos):
            self.A = amp
            self.u = pos
            self.sigma = width

        def __call__(self,x):
            return self.A/np.sqrt(2*np.pi)*np.exp(-(x-self.u)**2/(2*self.sigma**2))

    gauss_A = Gaussian_Func(amp_A,width_A,pos_A)
    gauss_I = Gaussian_Func(amp_I,width_I,pos_I)

    kernel_grid = int(grid*kernel_length/length)
    dist_array = np.zeros((kernel_grid,kernel_grid))

    center = kernel_grid/2
    for i in range(kernel_grid):
        for j in range(kernel_grid):
            r = (((i-center)*dx)**2+((j-center)*dx)**2)**0.5
            dist_array[i,j]=r

    #wheter a pattern can be generated or not
    kernel = gauss_A(dist_array)+gauss_I(dist_array)
    if np.abs(np.sum(kernel))>20:
        return False, None

    u1 = np.random.rand(grid,grid)
    u1_old = copy.deepcopy(u1)
    volume = np.sum(u1)

    lst = []
    flg = False

    diff_lst = []
    for t in range(loopNumber):
        for i in range(stepLoop):
            S = cv2.filter2D(u1,-1,kernel)
            S[S<0]=0
            S[S>max_S]=max_S
            u1 = u1+dt*(S-deg*u1)
            u1[u1<0]=0

            diff = np.mean(((u1_old -u1)**2)**0.5)
            diff_lst.append(diff)
            u1_old = copy.deepcopy(u1)

            if diff < 0.001:
                flg = True
                break
        print(diff)
        print(u1.max())
        lst.append(u1)
        h = copy.deepcopy(u1)
        h = h-h.min()
        h = h/h.max()*255
        h = h.astype(np.uint8)
        u2 = copy.deepcopy(u1)
        if flg:
            break

    if np.var(u1)>50:
        return h
    else:
        return False

In [ ]:
h = KT(amp_A,width_A,pos_A,amp_I,width_I,pos_I)
plt.imshow(h)
plt.show()